ICD10 code for HFpEF is 150.3 (diastolic)

ICD10 Code HFrEF I50.2 (systolic)

ICD9 code for HFpEF is 428.3

ICD9 Code HFrEF 428.2

https://physionet.org/content/mimiciv/2.2/hosp/diagnoses_icd.csv.gz

https://physionet.org/content/mimiciv/2.2/hosp/patients.csv.gz

https://physionet.org/content/mimiciv/2.2/hosp/admissions.csv.gz

labels =  1 for p, 0 for r

insurance = 1 for medicaid/medicare, 0 for private

In [ ]:
# patients from diagnosis, tag split, insurance, race, sex
# join with embedding file
# clean this data frame
# prep for model! (insurance, race, sex), label,	dicom_id,	embedding_file,	split,	image_id

# stats on patients (insurance, race, sex), class distribution ?

# precission, recall, f1, accuracy, AUC

In [ ]:
from google.colab import auth
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import os
from sklearn.decomposition import PCA
import os
from google.cloud import storage
import glob
from typing import Iterable
import functools
from typing import Dict, List, Optional, Tuple

In [ ]:
auth.authenticate_user()

In [ ]:
#links above

df_admissions = pd.read_csv('/content/admissions.csv.gz')[['subject_id', 'insurance', 'race']]
df_diagnoses = pd.read_csv('/content/diagnoses_icd.csv.gz')[['subject_id', 'icd_code', 'icd_version']]
df_patients = pd.read_csv('/content/patients.csv.gz')[['subject_id', 'gender']]

In [ ]:
icd9 = df_diagnoses[df_diagnoses['icd_version'] == 9]       # icd code is str
icd9['icd_code'] = icd9['icd_code'].str.slice(0,4)
icd9 = icd9[(icd9['icd_code'] == '4283') | (icd9['icd_code'] == '4282')]
icd9['label'] = icd9.apply(lambda x: 1 if x['icd_code'] == '4283' else 0, axis=1)
icd9

<ipython-input-20-73b40cfc81f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icd9['icd_code'] = icd9['icd_code'].str.slice(0,4)


,subject_id,icd_code,icd_version,label
252,10000980,4282,9,0
279,10000980,4283,9,1
330,10000980,4282,9,0
344,10000980,4283,9,1
631,10001877,4283,9,1
...,...,...,...,...
4755413,19998330,4283,9,1
4755429,19998330,4283,9,1
4755446,19998330,4283,9,1
4755463,19998330,4283,9,1


In [ ]:
icd10 = df_diagnoses[df_diagnoses['icd_version'] == 10]
icd10['icd_code'] = icd10['icd_code'].str.slice(0,4)
icd10 = icd10[(icd10['icd_code'] == 'I503') | (icd10['icd_code'] == 'I502')]
icd10['label'] = icd10.apply(lambda x: 1 if x['icd_code'] == 'I503' else 0, axis=1)
icd10

<ipython-input-21-1a30af8dc142>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icd10['icd_code'] = icd10['icd_code'].str.slice(0,4)


,subject_id,icd_code,icd_version,label
232,10000980,I503,10,1
308,10000980,I502,10,0
355,10000980,I502,10,0
574,10001667,I503,10,1
605,10001843,I503,10,1
...,...,...,...,...
4753647,19994379,I502,10,0
4755096,19997448,I503,10,1
4755110,19997473,I502,10,0
4755181,19997752,I503,10,1


In [ ]:
icd = icd9.append(icd10)[['subject_id', 'label']]
icd

<ipython-input-22-501340f409d9>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  icd = icd9.append(icd10)[['subject_id', 'label']]


,subject_id,label
252,10000980,0
279,10000980,1
330,10000980,0
344,10000980,1
631,10001877,1
...,...,...
4753647,19994379,0
4755096,19997448,1
4755110,19997473,0
4755181,19997752,1


In [ ]:
icd = icd.groupby(['subject_id']).aggregate({'label': list})
icd['label'] = icd['label'].str[0]
icd

,label
subject_id,
10000980,0
10001667,1
10001843,1
10001877,1
10002013,1
...,...
19997473,0
19997752,1
19997911,0


In [ ]:
agg_df = df_admissions.groupby('subject_id').aggregate({'insurance': list})
agg_df['insurance'] = agg_df.apply(lambda x: 1 if 'Medicaid' in x['insurance'] else 2 if 'Medicare' in x['insurance'] else 0, axis=1)
agg_df

,insurance
subject_id,
10000032,1
10000068,0
10000084,2
10000108,0
10000117,0
...,...
19999733,0
19999784,1
19999828,0


In [ ]:
agg_df.to_csv('medicaidcare.csv')

In [ ]:
df_labels = pd.read_csv('/content/data-labelled (2)')[['embedding_file', 'subject_id', 'study_id', 'dicom_id', 'insurance', 'race']]
df_labels = df_labels.merge(icd, on='subject_id', how='inner')
df_labels = df_labels.merge(df_patients,on='subject_id', how='inner')
df_labels = df_labels.merge(agg_df,on='subject_id', how='inner')
df_labels

,embedding_file,subject_id,study_id,dicom_id,insurance_x,race,label,gender,insurance_y
0,files/p10/p10000980/s50985099/6ad03ed1-97ee17e...,10000980,50985099,6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d,1.0,BLACK/AFRICAN AMERICAN,0,F,2
1,files/p10/p10000980/s51967283/943486a3-b3fa9ff...,10000980,51967283,943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4,1.0,BLACK/AFRICAN AMERICAN,0,F,2
2,files/p10/p10000980/s54577367/cfb03587-782edf6...,10000980,54577367,cfb03587-782edf6c-1bf392e1-98196cd5-365d69e8,1.0,BLACK/AFRICAN AMERICAN,0,F,2
3,files/p10/p10000980/s54935705/6ad819bb-bae74eb...,10000980,54935705,6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a,1.0,BLACK/AFRICAN AMERICAN,0,F,2
4,files/p10/p10000980/s54980801/a75a1fbe-802065a...,10000980,54980801,a75a1fbe-802065ad-717eb7c1-e2ce3552-646276a6,1.0,BLACK/AFRICAN AMERICAN,0,F,2
...,...,...,...,...,...,...,...,...,...
70655,files/p19/p19998330/s54878259/a0312c46-db14504...,19998330,54878259,a0312c46-db145040-b7bc5dc0-7bf8a213-393a3ab3,1.0,BLACK/AFRICAN AMERICAN,1,F,2
70656,files/p19/p19998330/s58626532/6c6f0868-10028db...,19998330,58626532,6c6f0868-10028db0-0270ff74-65210a42-e084c544,1.0,BLACK/AFRICAN AMERICAN,1,F,2
70657,files/p19/p19998330/s59281793/b7fea537-8782931...,19998330,59281793,b7fea537-87829311-389dc1c3-0947f202-bcf5f743,1.0,BLACK/AFRICAN AMERICAN,1,F,2
70658,files/p19/p19998330/s59694089/fb10d1e3-8298d0a...,19998330,59694089,fb10d1e3-8298d0a7-94438c16-66607b23-27127377,1.0,BLACK/AFRICAN AMERICAN,1,F,2


In [ ]:
temp = df_labels.groupby('subject_id').aggregate({'insurance_y':max})
temp.value_counts()

insurance_y
2              6058
0              2396
1               674
dtype: int64

In [ ]:
# map race
df_labels['race'] = df_labels['race'].map({'BLACK/AFRICAN AMERICAN': 'BLACK',
                                               'WHITE': 'WHITE / CAUCASION',
                                               'WHITE - RUSSIAN' : 'WHITE / CAUCASION',
                                               'PORTUGUESE' : 'OTHER',
       'BLACK/CAPE VERDEAN' : 'BLACK', 'ASIAN - CHINESE' : 'ASIAN', 'HISPANIC OR LATINO': 'HISPANIC / LATINO' ,
       'WHITE - OTHER EUROPEAN' : 'WHITE / CAUCASION', 'OTHER' : 'OTHER', 'BLACK/AFRICAN' : 'BLACK',
       'ASIAN - ASIAN INDIAN' : 'ASIAN', 'ASIAN' : 'ASIAN', 'UNKNOWN' : 'UNKNOWN',
       'WHITE - EASTERN EUROPEAN' : 'WHITE / CAUCASION', 'HISPANIC/LATINO - GUATEMALAN' : 'HISPANIC / LATINO',
       'PATIENT DECLINED TO ANSWER' : 'UNKNOWN', 'AMERICAN INDIAN/ALASKA NATIVE' : 'AMERICAN INDIAN / ALASKA NATIVE',
       'HISPANIC/LATINO - CENTRAL AMERICAN' : 'HISPANIC / LATINO', 'BLACK/CARIBBEAN ISLAND' : 'BLACK',
       'HISPANIC/LATINO - PUERTO RICAN' : 'HISPANIC / LATINO', 'UNABLE TO OBTAIN' : 'UNKNOWN',
       'HISPANIC/LATINO - DOMINICAN' : 'HISPANIC / LATINO', 'HISPANIC/LATINO - HONDURAN' : 'HISPANIC / LATINO',
       'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'AMERICAN INDIAN / ALASKA NATIVE',
       'HISPANIC/LATINO - MEXICAN' : 'HISPANIC / LATINO', 'ASIAN - SOUTH EAST ASIAN' : 'ASIAN',
       'WHITE - BRAZILIAN' : 'WHITE / CAUCASION', 'HISPANIC/LATINO - CUBAN' : 'HISPANIC / LATINO',
       'HISPANIC/LATINO - SALVADORAN' : 'HISPANIC / LATINO', 'MULTIPLE RACE/ETHNICITY' : 'OTHER',
       'SOUTH AMERICAN' :  'OTHER'})
df_labels

,embedding_file,subject_id,study_id,dicom_id,insurance,race,label,gender
0,files/p10/p10000980/s50985099/6ad03ed1-97ee17e...,10000980,50985099,6ad03ed1-97ee17ee-9cf8b320-f7011003-cd93b42d,1.0,BLACK,0,F
1,files/p10/p10000980/s51967283/943486a3-b3fa9ff...,10000980,51967283,943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4,1.0,BLACK,0,F
2,files/p10/p10000980/s54577367/cfb03587-782edf6...,10000980,54577367,cfb03587-782edf6c-1bf392e1-98196cd5-365d69e8,1.0,BLACK,0,F
3,files/p10/p10000980/s54935705/6ad819bb-bae74eb...,10000980,54935705,6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a,1.0,BLACK,0,F
4,files/p10/p10000980/s54980801/a75a1fbe-802065a...,10000980,54980801,a75a1fbe-802065ad-717eb7c1-e2ce3552-646276a6,1.0,BLACK,0,F
...,...,...,...,...,...,...,...,...
23002,files/p13/p13196638/s51956967/76947f29-410884d...,13196638,51956967,76947f29-410884d3-57ee9c25-f4781392-95ecd2e4,1.0,WHITE / CAUCASION,1,M
23003,files/p13/p13196638/s51956967/dc2f1215-d41959a...,13196638,51956967,dc2f1215-d41959af-0b4f5ae9-fd4480df-4d64625a,1.0,WHITE / CAUCASION,1,M
23004,files/p13/p13196638/s58056254/7f5a1c4f-4a9666e...,13196638,58056254,7f5a1c4f-4a9666ec-66008f60-16871c71-e69c0247,1.0,WHITE / CAUCASION,1,M
23005,files/p13/p13196638/s59401220/5dc626b5-c12038f...,13196638,59401220,5dc626b5-c12038f9-b5dea539-bfa56660-97787cef,1.0,WHITE / CAUCASION,1,M


In [ ]:
patient_demo = df_labels.groupby(['subject_id']).aggregate({'race': min, 'insurance': max, 'gender': max, 'label': max})
patient_demo

,race,insurance,gender,label
subject_id,,,,
10000980,BLACK,1.0,F,0
10002013,WHITE / CAUCASION,1.0,F,1
10002131,WHITE / CAUCASION,1.0,F,1
10002430,WHITE / CAUCASION,1.0,M,1
10003019,WHITE / CAUCASION,1.0,M,1
...,...,...,...,...
13194187,OTHER,0.0,M,0
13194374,WHITE / CAUCASION,0.0,F,0
13194394,WHITE / CAUCASION,0.0,M,0


In [ ]:
for c in patient_demo.columns:
    print("---- %s ---" % c)
    print(patient_demo[c].value_counts())

---- race ---
WHITE / CAUCASION                  2059
BLACK                               510
HISPANIC / LATINO                   124
OTHER                               103
UNKNOWN                              80
ASIAN                                79
AMERICAN INDIAN / ALASKA NATIVE      12
Name: race, dtype: int64
---- insurance ---
1.0    2194
0.0     773
Name: insurance, dtype: int64
---- gender ---
M    1496
F    1471
Name: gender, dtype: int64
---- label ---
1    1718
0    1249
Name: label, dtype: int64


In [ ]:
df_labels.to_csv('data-fin')
patient_demo.to_csv('patient_demo')